In [2]:
#This is a modification of the original script for Google Colab
#This script is used to download and run a Minecraft server on a GitHub Codespace
#By @Lucast1574 @lucast_1574
# Just versions +1.17 are available
# AVAILABLE TESTED VERSIONS:
# - 1.20.4
# - 1.20.3
# - 1.20.2
# - 1.20.1
# - 1.20
# - 1.19.4
# - 1.19.3
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1



version = '1.20.1' # CHANGE THIS TO THE VERSION YOU WANT TO USE

#Chose server type. Currently available versions:
# - paper (The most optimized server for Spigot/Bukkit)
# - forge (For modded servers)
# - fabric (For modded servers)(Alternative to forge)
# - vanilla (For vanilla servers)
# - snapshot (For testing new features)
# - mohist (For modded servers and plugins) 1.20.1 29/09/24 last version available
# - catserver (For modded servers and plugins) 1.18.2 29/09/24 last version available
# - purpur (For modded servers with fabric and plugins)

# To use paper, you must install vanilla first and then install paper
# To use mohist or catserver, you must install forge first and then install mohist or catserver
# To use purpur, you must install fabric first and then install purpur

server_type = 'paper'

#Here we import the necessary libraries

import os
import requests
import json


# Name of the server directory

server_directory = "Minecraft-server"

# Create the server directory on the codespace

os.makedirs(server_directory, exist_ok=True)
os.chdir(server_directory)

# Internal init...

if server_type == 'paper':
    a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)

    b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))

    print("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"])
    serverURL = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]

if server_type == 'forge':
    serverURL = "https://serverjars.com/api/fetchJar/modded/forge/" + version

if server_type == 'vanilla':
    serverURL = "https://serverjars.com/api/fetchJar/vanilla/vanilla/" + version

if server_type == 'snapshot':
    serverURL = "https://serverjars.com/api/fetchJar/vanilla/snapshot/" + version

if server_type == 'fabric':
    serverURL = 'https://serverjars.com/api/fetchJar/modded/fabric/' + version

if server_type == 'mohist':
    serverURL = "https://serverjars.com/api/fetchJar/modded/mohist/" + version
    
if server_type == 'catserver':
    serverURL = "https://serverjars.com/api/fetchJar/modded/catserver/" + version

if server_type == 'purpur':
    serverURL = "https://serverjars.com/api/fetchJar/servers/purpur/" + version


# Download the server jar based on the server type

jar_name = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': "snapshot.jar", 'mohist': "mohist.jar", 
            'catserver': "catserver.jar", 'purpur': "purpur.jar"}

print('Downloading the selected server: ')

if serverURL:
    print('Dowloading to github codespaces...')
    r = requests.get(serverURL)

    if r.status_code == 200:
        with open(jar_name[server_type], 'wb') as f:
            f.write(r.content)
    else:
        print('Error '+ str(r.status_code) + '! The version you choose does not work.')


# Run on a specific path depending on the server type

# If the server is fabric, we need to run the installer

if server_type == 'fabric':
    !java -jar fabric-server-launch.jar server -mcversion $version -downloadMinecraft

# If the server is forge, we need to run the installer
if server_type == 'forge':
    %cd "/workspaces/codespaces-jupyter/Minecraft-server"
    !java -jar forge.jar --installServer

# If the server is vanilla, we need to run the installer

elif server_type == 'vanilla':
    %cd "/workspaces/codespaces-jupyter/Minecraft-server"
    !java -jar vanilla.jar --installServer
# Saves the configuration file on the server
colabconfig = {"server_type": server_type, "server_version": version}
with open("colabconfig.json", 'w') as config_file:
    json.dump(colabconfig, config_file)

# print a completion message
print('Done!')
# change eula to true
!echo "eula=true" >> eula.txt


https://papermc.io/api/v2/projects/paper/versions/1.20.1/builds/196/downloads/paper-1.20.1-196.jar
Dowloading to github codespaces...
Done!


In [1]:
#You have to put the spawn protection in 0 so you can play normally(this inside Server.properties)
#import the libraries needed for the server to run

import requests

import time

import os

import re

import json

import glob

import threading

import subprocess


#Update apt cache

!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"

#Asign the files to the directory

os.makedirs("/workspaces/codespaces-jupyter/Minecraft-server", exist_ok=True)

os.chdir("/workspaces/codespaces-jupyter/Minecraft-server")

!ls #List the files in the directory to verify them



#Import the configuration file

config_path = "/workspaces/codespaces-jupyter/Minecraft-server/colabconfig.json"

if os.path.isfile(config_path):

    with open(config_path) as config_file:

        colabconfig = json.load(config_file)

else:

    colabconfig = {"server_type": "generic"} # Use the default configuration if there the configuration file does not exist

    with open(config_path, 'w') as new_config_file:

        json.dump(colabconfig, new_config_file)



# Instal OpenJDK

version = colabconfig["server_version"]

# Execute a command and check if it was successful

def execute_command(comand):
    process = subprocess.run(comand, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Check if the process was successful
    if process.returncode == 0:
        print("OpenJDK 17 installed")
    else:
        print("Failed to install OpenJDK 17")

# Purge existing OpenJDK installations
execute_command('sudo apt-get purge openjdk* -y')

# Install OpenJDK 17
execute_command('sudo apt-get install openjdk-17-jre-headless -y')
    

# Check the java version

java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'

#List of the server types and their respective jar files

jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': 'snapshot.jar', 'mohist': "mohist.jar", 'catserver': "catserver.jar", 'purpur': "purpur.jar"}

jar_name = jar_list[colabconfig["server_type"]]




# Java arguments.

if colabconfig["server_type"] == "paper":

    server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"

else:

    server_flags = "" # No flags for other server types

#JVM Arguments for the server

memory_allocation = "-Xms10G -Xmx10G"



# tunnel_service for the server

tunnel_service = "ngrok"

print("Using: ", tunnel_service)


if tunnel_service == "ngrok":

    !pip -q install pyngrok #Here you instal pyngrok to use ngrok on the codespace

    from pyngrok import conf, ngrok



    # get your ngrok token from https://dashboard.ngrok.com/auth

    print("Get your authtoken from: https://dashboard.ngrok.com/auth") #Here you get your authtoken from ngrok, it will be a dinamic token, if you want a static token you need to pay for ngrok

    import getpass



    # v - - - - - - - TOKEN - - - - - - - v



    authtoken = "2dFiCwvYiy5sK9sAEfHnduhTHzH_2tt3vymbciAGp1anyCt82" # <---- Place yout token here

    ! ngrok authtoken $authtoken # log in ngrok



    #Choose your region



    #v - - - - - - - Available Regions - - - - - - - v

    # ap - Asia/Pacific (Singapore)

    # au - Australia (Sydney)

    # eu - Europa (Frankfurt - Alemania)

    # in - India (Mumbai)

    # jp - Japon (Tokyo)

    # sa - South america (São Paulo - Brasil)

    # us - United estates (Ohio)

    conf.get_default().region = 'ap' # <--- Place your region here


    # Conect to ngrok

    url = ngrok.connect(25565, 'tcp')

    print('Your server IP: ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

    #If you got the premiun version of ngrok use the comented code below to get a subdomain

    #subdomain = "my-subdomain" # <--- Place your subdomain here inside the quotes

    #url = ngrok.connect(25565, 'tcp', subdomain=subdomain)

    #print('Your server ip is:  ' + url.replace('tcp://', ''))

#If using playit.gg

elif tunnel_service == "playit":

    # Install playit.gg
    ! curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -
    
    # Add playit.gg repository
    
    ! sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list
    
    # Install playit.gg
    
    ! sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
    
    print('Starting server...')
    
    # Start the server
    
    ! playit & java $memory_allocation $server_flags -jar $jar_name nogui

print('Launching server...')

def get_users():
    url = 'https://jsonplaceholder.typicode.com/users'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            print("Lista de usuarios:")
            for user in data:
                print(f"Name: {user['name']}, Email: {user['email']}")
        else:
            print(f"Error making the request. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Conexion Error: {e}")

# Funcion for executing get_users() every 10 minutes

def repeat_get_users():
    while True:
        get_users()
        time.sleep(600)  # Wait 10 minutes

# Start the thread for repeat_get_users()

user_thread = threading.Thread(target=repeat_get_users)
user_thread.start()

if colabconfig["server_type"] == "forge":

    version = colabconfig["server_version"]
    
    forgefiles = f"/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/"
    
    forgepreversion = os.listdir(forgefiles)
    
    if forgepreversion:
    
        forgeversion = forgepreversion[0]
        forgeversionchecked = forgeversion.replace(".jar", "")
        print("Your forge version is: ", forgeversionchecked)
    
    else:
    
        print("There was not any forge files.")
    
    pathtoforge = glob.glob(f"/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/{forgeversionchecked}/unix_args.txt")

    if pathtoforge: 

        path = pathtoforge[0] 

        print(path)

        !java @user_jvm_args.txt "@{path}" "$@"

    else:

        print("No unix_args.txt found.")


else:
    
    !java $memory_allocation $server_flags -jar $jar_name nogui   


# Cicle to keep the server running

while True:
    time.sleep(24000) #Wait 60 seconds 4800=1h

apt cache successfully updated
Minecraft-server     config	      plugins		    versions
banned-ips.json      eula.txt	      server.jar	    wepif.yml
banned-players.json  help.yml	      server.properties     whitelist.json
bukkit.yml	     libraries	      spigot.yml	    world
cache		     logs	      usercache.json	    world_nether
colabconfig.json     ops.json	      usernamecache.json    world_the_end
commands.yml	     permissions.yml  version_history.json
OpenJDK 17 installed
OpenJDK 17 installed
Using:  ngrok
Get your authtoken from: https://dashboard.ngrok.com/auth
Authtoken saved to configuration file: /home/codespace/.config/ngrok/ngrok.yml
Your server IP: 0.tcp.ap.ngrok.io:10094
Launching server...
Lista de usuarios:
Name: Leanne Graham, Email: Sincere@april.biz
Name: Ervin Howell, Email: Shanna@melissa.tv
Name: Clementine Bauch, Email: Nathan@yesenia.net
Name: Patricia Lebsack, Email: Julianne.OConner@kory.org
Name: Chelsey Dietrich, Email: Lucio_Hettinger@annie.ca
Name: Mrs. Dennis